<a href="https://colab.research.google.com/github/LeeSeungHyun4839/ML-Simulator/blob/master/ML_simulator_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

BATCH_SIZE = 128
LEARNING_RATE = 0.002
EPOCHS = 1000
SNR = 25
I = 3

In [ ]:
import csv
import numpy as np
channel_true_Re = []
with open('./SCM_channel_true_Re.csv') as csvfile:
  Channel_true_Re_temp = csv.reader(csvfile, delimiter = ",")
  for row in Channel_true_Re_temp:
    channel_true_Re.append((row))
channel_true_Re = np.array(channel_true_Re, float)

channel_true_Im = []
with open('./SCM_channel_true_Im.csv') as csvfile:
  Channel_true_Im_temp = csv.reader(csvfile, delimiter = ",")
  for row in Channel_true_Im_temp:
    channel_true_Im.append((row))
channel_true_Im = np.array(channel_true_Im, float)

print(np.shape(channel_true_Im))

(2500, 64)


In [ ]:
filename_Re = './SCM_channel_SNR' + str(SNR) + '_Re.csv'
filename_Im = './SCM_channel_SNR' + str(SNR) + '_Im.csv'

channel_SNR_Re = []
with open(filename_Re) as csvfile:
  Channel_SNR_Re_temp = csv.reader(csvfile, delimiter = ",")
  for row in Channel_SNR_Re_temp:
    channel_SNR_Re.append((row))
channel_SNR_Re = np.array(channel_SNR_Re, float)

channel_SNR_Im = []
with open(filename_Im) as csvfile:
  Channel_SNR_Im_temp = csv.reader(csvfile, delimiter = ",")
  for row in Channel_SNR_Im_temp:
    channel_SNR_Im.append((row))
channel_SNR_Im = np.array(channel_SNR_Im, float)

In [ ]:
# Training data set
train = []

Train_Re = np.reshape(channel_SNR_Re[:2048, :], (2048,8,8))
Train_Im = np.reshape(channel_SNR_Im[:2048, :], (2048,8,8))

for i in range(2048-I):
  train_re = Train_Re[i:I+i,:,:]
  train_im = Train_Im[i:I+i,:,:]

  train_i = np.concatenate((train_re, train_im))
  train.append(train_i)

train = np.array(train)

label = np.concatenate((channel_SNR_Re[I:2048, :], \
                             channel_SNR_Im[I:2048, :]), axis = 1)

print(np.shape(train))
print(np.shape(label))

(2045, 6, 8, 8)
(2045, 128)


In [ ]:
# Test data set
Test_Re = np.reshape(channel_SNR_Re[2200-I:2299, :], (99+I,8,8))
Test_Im = np.reshape(channel_SNR_Im[2200-I:2299, :], (99+I,8,8))

test = []

for i in range(100):
  test_re = Test_Re[i:I+i,:,:]
  test_im = Test_Im[i:I+i,:,:]

  test_i = np.concatenate((test_re, test_im))
  test.append(test_i)

test = np.array(test)

label_test = np.concatenate((channel_true_Re[2200:2300, :], \
                             channel_true_Im[2200:2300, :]), axis = 1)

print(np.shape(test))
print(np.shape(label_test))

(100, 6, 8, 8)
(100, 128)


In [ ]:
test = torch.from_numpy(test).float()
label_test = torch.from_numpy(label_test).float()

In [ ]:
train = torch.from_numpy(train).float()
label = torch.from_numpy(label).float()

dataset_train = TensorDataset(train, label)
data_loader = DataLoader(dataset_train, batch_size = BATCH_SIZE, shuffle = True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(I*2, 16, kernel_size = 3, padding = 'same')
    self.conv2 = nn.Conv2d(16, 16, kernel_size = 3, padding = 'same')
    self.bn = nn.BatchNorm2d(16)
    self.fc = nn.Linear(8 * 8 * 16, 128, bias = True)
    
  def forward(self, input):
    output = self.conv1(input)
    output = self.bn(output)
    output = self.conv2(output)
    output = self.bn(output)
    output = output.view(output.size(0), -1) # flatten
    output = self.fc(output)

    return output

In [ ]:
model = CNN().to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
loss_ft = nn.MSELoss().to(device)

for epoch in range(EPOCHS):
  losses = []

  for X_batch, Y_batch in data_loader:
    X_batch = X_batch.to(device)
    Y_batch = Y_batch.to(device)
    optimizer.zero_grad()
    pred = model(X_batch)
    loss = loss_ft(pred, Y_batch)
    loss.backward()
    optimizer.step()

    losses.append(loss.cpu().detach().numpy())

  print(f'Epoch {epoch+1:3d} :: Loss is {np.array(losses).mean():.6f}')


Epoch   1 :: Loss is 0.086435
Epoch   2 :: Loss is 0.013608
Epoch   3 :: Loss is 0.004457
Epoch   4 :: Loss is 0.002130
Epoch   5 :: Loss is 0.001452
Epoch   6 :: Loss is 0.001202
Epoch   7 :: Loss is 0.001264
Epoch   8 :: Loss is 0.001434
Epoch   9 :: Loss is 0.001664
Epoch  10 :: Loss is 0.001897
Epoch  11 :: Loss is 0.001621
Epoch  12 :: Loss is 0.001481
Epoch  13 :: Loss is 0.001549
Epoch  14 :: Loss is 0.001456
Epoch  15 :: Loss is 0.001367
Epoch  16 :: Loss is 0.001332
Epoch  17 :: Loss is 0.001226
Epoch  18 :: Loss is 0.001283
Epoch  19 :: Loss is 0.001338
Epoch  20 :: Loss is 0.001284
Epoch  21 :: Loss is 0.001799
Epoch  22 :: Loss is 0.001868
Epoch  23 :: Loss is 0.001486
Epoch  24 :: Loss is 0.001133
Epoch  25 :: Loss is 0.001558
Epoch  26 :: Loss is 0.001740
Epoch  27 :: Loss is 0.001322
Epoch  28 :: Loss is 0.001054
Epoch  29 :: Loss is 0.001322
Epoch  30 :: Loss is 0.001075
Epoch  31 :: Loss is 0.000901
Epoch  32 :: Loss is 0.000901
Epoch  33 :: Loss is 0.000818
Epoch  34 

In [ ]:
with torch.no_grad():
    test = test.to(device)
    label_test = label_test.to(device)
    prediction = model(test)
    h = label_test
    h_hat = prediction
    error = 0
    for i in range(100):
        h_i = h[i,:]
        h_hat_i = h_hat[i,:]
        error += torch.sum((h_i-h_hat_i)**2) / torch.sum((h_i)**2) /100
    
    error = error.float()
    print('NMSE =', error)

    dBerror = 10 * torch.log(error)

    print('Decibel NMSE =', dBerror)

NMSE = tensor(0.0064, device='cuda:0')
Decibel NMSE = tensor(-50.5029, device='cuda:0')
